In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [79]:
df=pd.read_csv("D:\\datasets\\archive (1)\\PlayTennis.csv")

In [59]:
df.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Outlook      14 non-null     object
 1   Temperature  14 non-null     object
 2   Humidity     14 non-null     object
 3   Wind         14 non-null     object
 4   Play Tennis  14 non-null     object
dtypes: object(5)
memory usage: 688.0+ bytes


In [61]:
df.isnull().sum()

Outlook        0
Temperature    0
Humidity       0
Wind           0
Play Tennis    0
dtype: int64

In [62]:
from sklearn import preprocessing
my_label=preprocessing.LabelEncoder()
df['Outlook']=my_label.fit_transform(df['Outlook'])

df['Temperature']=my_label.fit_transform(df['Temperature'])

df['Humidity']=my_label.fit_transform(df['Humidity'])

df['Wind']=my_label.fit_transform(df['Wind'])

df['Play Tennis']=my_label.fit_transform(df['Play Tennis'])

In [63]:
column = 'Outlook'
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min()) 
column = 'Temperature'
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min()) 
column = 'Humidity'
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min()) 
column = 'Wind'
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min()) 
column = 'Play Tennis'
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min()) 

In [80]:
from operator import itemgetter
import math

In [81]:
class DecisionTree:
    def __init__(self, df, target, positive, parent_val, parent):
        self.data = df
        self.target = target
        self.positive = positive
        self.parent_val = parent_val
        self.parent = parent
        self.childs = []
        self.decision = ''

    def _get_entropy(self, data):
        p = sum(data[self.target]==self.positive)
        n = data.shape[0] - p
        p_ratio = p/(p+n)
        n_ratio = 1 - p_ratio
        entropy_p = -p_ratio*math.log2(p_ratio) if p_ratio != 0 else 0
        entropy_n = - n_ratio*math.log2(n_ratio) if n_ratio !=0 else 0
        return entropy_p + entropy_n
    
    def _get_gain(self, feat):
        avg_info=0
        for val in self.data[feat].unique():
            avg_info+=self._get_entropy(self.data[self.data[feat] == val])*sum(self.data[feat]==val)/self.data.shape[0]
        return self._get_entropy(df) - avg_info
    
    def _get_splitter(self):
        self.splitter = max(self.gains, key = itemgetter(1))[0] 
    
    def update_nodes(self):
        self.features = [col for col in self.data.columns if col != self.target]
        self.entropy = self._get_entropy(self.data)
        if self.entropy != 0:
            self.gains = [(feat, self._get_gain(feat)) for feat in self.features]
            self._get_splitter()
            residual_columns = [k for k in self.data.columns if k != self.splitter]
            for val in self.data[self.splitter].unique():
                df_tmp = self.data[self.data[self.splitter]==val][residual_columns]
                tmp_node = DecisionTree(df_tmp, self.target, self.positive, val, self.splitter)
                tmp_node.update_nodes()
                self.childs.append(tmp_node)

In [82]:
def print_tree(n):
    for child in n.childs:
        if child:
            print(child.__dict__.get('parent', ''))
            print(child.__dict__.get('parent_val', ''), '\n')
            print_tree(child)

In [83]:
dt = DecisionTree(df, 'Play Tennis', 'Yes', '', '')
dt.update_nodes()
print_tree(dt)

Outlook
Sunny 

Humidity
High 

Humidity
Normal 

Outlook
Overcast 

Outlook
Rain 

Wind
Weak 

Wind
Strong 

